# SWE546_Assignment3

## Nearest Neighbour

Implement the k nearest neighbor algorithm and measure the accuracy as well as precision and recall for the classification of iris type 2 versus 1,3.

*Submit a jupyter notebook.*

**Student Name: Talat Aydın Çıkıkcı**

### Contents:

* [Initialization](#Initialization)
* [Classify 1 versus 2,3](#Classify-1-versus-2,3)
* [Classify 2 versus 1,3](#Classify-2-versus-1,3)
* [Classify 3 versus 1,2](#Classify-3-versus-1,2)

## Solution

### Initialization

In [10]:
%matplotlib inline

# Import required libraries
import scipy as sc
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import heapq as hp

# Get data from data file
df_iris = pd.read_csv(u'data/iris.txt',sep=' ')
df_iris

sl   sw   pl   pw  c
0    5.1  3.5  1.4  0.2  1
1    4.9  3.0  1.4  0.2  1
2    4.7  3.2  1.3  0.2  1
3    4.6  3.1  1.5  0.2  1
4    5.0  3.6  1.4  0.2  1
5    5.4  3.9  1.7  0.4  1
6    4.6  3.4  1.4  0.3  1
7    5.0  3.4  1.5  0.2  1
8    4.4  2.9  1.4  0.2  1
9    4.9  3.1  1.5  0.1  1
10   5.4  3.7  1.5  0.2  1
11   4.8  3.4  1.6  0.2  1
12   4.8  3.0  1.4  0.1  1
13   4.3  3.0  1.1  0.1  1
14   5.8  4.0  1.2  0.2  1
15   5.7  4.4  1.5  0.4  1
16   5.4  3.9  1.3  0.4  1
17   5.1  3.5  1.4  0.3  1
18   5.7  3.8  1.7  0.3  1
19   5.1  3.8  1.5  0.3  1
20   5.4  3.4  1.7  0.2  1
21   5.1  3.7  1.5  0.4  1
22   4.6  3.6  1.0  0.2  1
23   5.1  3.3  1.7  0.5  1
24   4.8  3.4  1.9  0.2  1
25   5.0  3.0  1.6  0.2  1
26   5.0  3.4  1.6  0.4  1
27   5.2  3.5  1.5  0.2  1
28   5.2  3.4  1.4  0.2  1
29   4.7  3.2  1.6  0.2  1
..   ...  ...  ...  ... ..
120  6.9  3.2  5.7  2.3  3
121  5.6  2.8  4.9  2.0  3
122  7.7  2.8  6.7  2.0  3
123  6.3  2.7  4.9  1.8  3
124  6.7  3.3  5.7  2.1  3
125  7.2  3.2  6.0  1.8  3
126  6.2  2.8  4.8  1.8  3
127  6.1  3.0  4.9  1.8  3
128  6.4  2.8  5.6  2.1  3
129  7.2  3.0  5.8  1.6  3
130  7.4  2.8  6.1  1.9  3
131  7.9  3.8  6.4  2.0  3
132  6.4  2.8  5.6  2.2  3
133  6.3  2.8  5.1  1.5  3
134  6.1  2.6  5.6  1.4  3
135  7.7  3.0  6.1  2.3  3
136  6.3  3.4  5.6  2.4  3
137  6.4  3.1  5.5  1.8  3
138  6.0  3.0  4.8  1.8  3
139  6.9  3.1  5.4  2.1  3
140  6.7  3.1  5.6  2.4  3
141  6.9  3.1  5.1  2.3  3
142  5.8  2.7  5.1  1.9  3
143  6.8  3.2  5.9  2.3  3
144  6.7  3.3  5.7  2.5  3
145  6.7  3.0  5.2  2.3  3
146  6.3  2.5  5.0  1.9  3
147  6.5  3.0  5.2  2.0  3
148  6.2  3.4  5.4  2.3  3
149  5.9  3.0  5.1  1.8  3

[150 rows x 5 columns]

In [11]:
# Seperate data according to columns.
sepal_l = np.matrix(df_iris.sl[0:]).T
sepal_w = np.matrix(df_iris.sw[0:]).T
petal_l = np.matrix(df_iris.pl[0:]).T
petal_w = np.matrix(df_iris.pw[0:]).T

X = np.hstack((sepal_l, sepal_w, petal_l, petal_w))

# Initialize "iris_class"
iris_class = np.matrix(df_iris.c[0:]).T

# Define class with comparison properties.
class heapClass:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)

In [19]:
## Search for the nearest K   
K = 20
heap = []
N = X.shape[0]   

for j in range(N):
    heap = []
    ## Fill in the heap with dummy nodes
    for k in range(K):
        hp.heappush(heap, heapClass(np.inf, -1))

    for i in range(N):
        if i != j:
            e = X[i,:]- X[j,:]
            e = e.reshape((4,1))
            tp = heapClass(float(e.T*e), i)
            if tp <= heap[0]:
                hp.heapreplace(heap, tp)

    for t in range(len(heap)):
        h = hp.heappop(heap)
        print(float(iris_class[h.idx]))
    print('--'*5)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
----------